# Practical Exercise 4.01: Bond Characteristics

In [ ]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
from tabulate import tabulate


In [ ]:
#Downloading IRS Curve from a file

infile=pd.ExcelFile("EURIBORISR Reuters.xlsx")
IRSwaps=infile.parse('Hoja2')

#IRS file filtering

IRSwaps=IRSwaps[['Term','Ask']]
IRSwaps=IRSwaps.rename(columns={'Ask':'SWRate'})
IRSwaps.index+=1

# Zero coupon interest rates in the interbank market are constructed from IRS rates.

ZCC=[] # A matrix is created to collect the zero coupon interest rates.
i1=IRSwaps.SWRate[1]/100 # The 1-year zero coupon rate is matched to the 1-year IRS rate.

ZCC.append(i1)

for x in range(2,21): # The zero coupon rates for years 2 to 20 are calculated.

    maturity=x # Rates will be calculated by maturity.
    Coupon=[] # The matrix that will collect the coupons is created.
    for n in range(maturity-1): # The coupons of all maturities are selected except for the last year.

        Maturities=list(range(maturity)) # A maturity list is created
        coupon=(IRSwaps.SWRate[maturity])/((1+ZCC[n])**(Maturities[n+1]))
        # Coupons are updated to the zero coupon rates generated up to that time.
        Coupon.append(coupon) # The updated coupons are added to the coupon matrix.
        VACoupon=np.sum(Coupon) # The total value of the updated coupons is calculated

        Current_Price=100-(VACoupon) # To find the price at source, the present value of the coupons is subtracted from the nominal value.

        Last_CashFlow=100+IRSwaps.SWRate[maturity] # The last cash flow of the bond is calculated, which is the nominal plus the coupon.

        i1=((Last_CashFlow/Current_Price)**(1/maturity))-1 #The zero coupon rate is calculated for this maturity.

    ZCC.append(i1) # The zero coupon rate is added to those already calculated for previous periods.


ZCC=[round(b*100,5) for b in ZCC] # Rates are rounded
ZCC=pd.DataFrame(ZCC)

# The comparison between swap rates and interbank zero-coupon rates is displayed.

ZCCIntbank=pd.DataFrame(ZCC)
ZCCIntbank=ZCCIntbank.rename(columns={0:'ZCRate'})
ZCCIntbank.index+=1
Rates=pd.concat([IRSwaps,ZCCIntbank], axis=1)
Rates



In [ ]:
# Both curves are plotted graphically

plt.figure(figsize=(12, 8))
plt.grid(True)
plt.scatter(Rates['Term'], Rates['ZCRate'] ,marker='o',color='r',s=50, label='Zero Coupon Rates')
plt.plot(Rates['Term'], Rates['ZCRate'] ,linestyle='-', color='black', label='Zero Coupon Curve')
plt.scatter(Rates['Term'], Rates['SWRate'] ,marker='o',color='b',s=50, label='Swaps Rates')
plt.plot(Rates['Term'], Rates['SWRate'] ,linestyle='-', color='grey', label='Swaps Curve')
plt.title('ZERO COUPON CURVE versus SWAPS CURVE')
plt.xlabel('YEARS')
plt.ylabel('RATES (%)')
plt.legend(labelspacing=0.8)


In [ ]:
# We define the bond characteristics function

def bond_characteristics_calculation(face_value, mty, coupons, SWsprd):

    # Calculation of discount factor

    ZCRates = 1 + (Rates.ZCRate.T / 100) + (SWsprd / 10000)


    # Calculation of the present value of all cash flows

    coupon=(coupons/100)*face_value
    coupons_record = np.array([coupon] * mty)

    # Generation of maturities

    n = np.arange(1, mty + 1)

    # Value of each coupon discounted at the discount factor raised at maturity

    VAcoupons = coupons_record/(ZCRates.values[0:mty].T ** n)
    SUMvacoupons = np.sum(VAcoupons) # Sum of all coupons at current value



    # Calculation of the present value of the nominal value

    VAFaceVal = face_value / (ZCRates.values[(mty - 1):mty] ** mty)


    # Bond price calculation

    BondPrice = (SUMvacoupons + VAFaceVal).round(2)
    BondPrice = BondPrice.item()


    # Establishing the cash flows of the Bond

    BondCF=list(coupons_record[0:mty - 1])
    BondCF.append(face_value+(coupon))
    BondCF.insert(0, -BondPrice)

   # Transforming cash flows to normal floats
    BondCF = [float(x) for x in BondCF]


    # Calculating the bond's IRR from its cash flows.

    BondIRR = npf.irr(BondCF) * 100

    # Calculating the MacAuley duration of the bond

    Maturities = np.arange(1, mty)
    VAFace = (face_value + coupon) / (ZCRates.values[(mty - 1):mty] ** mty)
    CFDur = [(a * b) for a, b in zip(VAcoupons, Maturities)]
    SUMACFDur = np.sum(CFDur)
    Duration = (SUMACFDur + (VAFace * mty)) / BondPrice
    Duration = Duration.item()
    ModifiedDuration = Duration / (1 + (BondIRR / 100))

    # Calculate convexity

    t = np.arange(1, mty)
    VAcoupons2 = coupons_record[1:mty + 1] / ((1 + (BondIRR / 100)) ** (t + 2))
    Maturities2 = [i * (i + 1) for i in range(1, mty)]
    CFConvx = [(a * b) for a, b in zip(VAcoupons2, Maturities2)]
    VAFaceConvx = (face_value + coupon) / ((1 + (BondIRR / 100)) ** (mty + 2))
    VAFaceConvx = VAFaceConvx * mty * (mty + 1)
    AbsConvx = np.sum(CFConvx) + VAFaceConvx
    ModConvx = AbsConvx / BondPrice
    ModConvx = ModConvx.item()

    # Creating a results dictionary

    results = {
        'Metric': ['Face Value (€)', 'Bond Price (€)', 'Bond Maturity (years)', 'Coupon (€)','Coupon rate (%)','Bond IRR (%)', 'Duration (years)', 'Modified Duration', 'Discrete Convexity', 'Bond Cash-Flows'],
        'Value': [round(face_value,2), BondPrice, len(Maturities)+1, coupon, coupons, round(BondIRR,2), round(Duration,2), round(ModifiedDuration,2),  round(ModConvx,2), BondCF]
    }


    # Converting dictionary to a DataFrame

    results_df = pd.DataFrame(results)

    return results_df

def printing_in_bold_type(df):
    bold_end = '\033[1m'
    table = tabulate(df, headers='keys', tablefmt='tsv')
    bold_start = '\033[1m'
    table = table.replace('|', f'{bold_start}|{bold_end}')
    table = table.replace('-', f'{bold_start}-{bold_end}')
    return f'{bold_start}{table}{bold_end}'


In [ ]:
# Apply for the above-mentioned bond

face_value=1000
mty=10 # Maturity of the Bond, in years
coupons=2.00 # Bond coupon, as a percentage of par value
SWsprd=50 # Bond spread over the interbank zero-coupon curve, in basis points

bond_characteristics= bond_characteristics_calculation(face_value, mty, coupons, SWsprd)
bond_characteristics.index+=1

print(printing_in_bold_type(bond_characteristics))
